In [1]:
# built-in imports
!pip install requests
!pip install speechrecognition==3.8.1
!pip install paramiko

from naoqi import ALProxy
import time
import threading
import requests
import json
import os
import speech_recognition as sr
import paramiko

DEPRECATION: Python 2.7 reached the end of its life on January 1st, 2020. Please upgrade your Python as Python 2.7 is no longer maintained. A future version of pip will drop support for Python 2.7. More details about Python 2 support in pip, can be found at https://pip.pypa.io/en/latest/development/release-process/#python-2-support
You should consider upgrading via the '/usr/local/bin/python -m pip install --upgrade pip' command.
DEPRECATION: Python 2.7 reached the end of its life on January 1st, 2020. Please upgrade your Python as Python 2.7 is no longer maintained. A future version of pip will drop support for Python 2.7. More details about Python 2 support in pip, can be found at https://pip.pypa.io/en/latest/development/release-process/#python-2-support
You should consider upgrading via the '/usr/local/bin/python -m pip install --upgrade pip' command.
DEPRECATION: Python 2.7 reached the end of its life on January 1st, 2020. Please upgrade your Python as Python 2.7 is no longer main

/usr/local/lib/python2.7/site-packages/paramiko/transport.py:33: CryptographyDeprecationWarning: Python 2 is no longer supported by the Python core team. Support for it is now deprecated in cryptography, and will be removed in the next release.
  from cryptography.hazmat.backends import default_backend


## Constants

In [2]:
GPT_HOST = "10.104.23.208"
ip = PEPPER_HOST = "10.104.23.185"

GPT_PORT = 8891
port = PEPPER_PORT = 9559

## Tablet

In [9]:
from time import time, sleep

tabletService = ALProxy("ALTabletService", ip, port)
tabletService.loadUrl('http://198.18.0.1/page.html')
tabletService.showWebview()

True

In [ ]:
receive_file('',)

## Audio Recording

In [3]:
def eye_color(status="ListenOn", rgb = None):
    leds = ALProxy("ALLeds", ip, port)

    if rgb is None:
        if status == "ListenOn":
            leds.fadeRGB("AllLeds", "yellow", 1)
        elif status == "SpeechDetected":
            leds.fadeRGB("AllLeds", "green", 8)
        elif status == "EndOfProcess":
            leds.fadeRGB("AllLeds", "white", 1)

    else:
        leds.fadeRGB("FaceLeds", rgb, 1)


# Callback function for timer
def timer_callback(timer_cb):
    print("Timer reached")
    timer_cb.set()

# Records Audio when speech is detected
def record_audio(timer=None, path_name="/home/nao/microphones/recording.wav", debug=False):
    path_name = os.path.join(path_name)
    # Create Connection Proxies
    recorder = ALProxy("ALAudioRecorder", ip, port)
    speech_recogniser = ALProxy("ALSpeechRecognition", ip, port)
    sound_detector = ALProxy("ALSoundDetection", ip, port)
    memory = ALProxy("ALMemory", ip, port)

    speech_recogniser.subscribe("speech_recognition")
    speech_recogniser.setLanguage("English")
    vocabulary = ["pepper", "hello"]
    speech_recogniser.setVocabulary(vocabulary, False)

    # Stop recording when speech stops being detected or timer is reached
    timer_cb = threading.Event()
    if timer is not None:
        threading.Timer(timer, timer_callback, timer_cb).start()

    # Start recording
    while True:
        time.sleep(0.5)
        status = memory.getData("ALSpeechRecognition/Status")

        # Print status
        if debug:
            print(status)

        # Start recording when speech is detected
        if status == "SpeechDetected":
            eye_color(status)
            print("Recording is starting...")
            recorder.startMicrophonesRecording(path_name, "wav", 16000, (0,0,1,0))
            break

        # Stop recording when timer is reached
        if timer_cb.is_set():
            break

    if timer_cb.is_set():
        return

    while True:
        time.sleep(0.5)
        status = memory.getData("ALSpeechRecognition/Status")

        # Stop recording when speech stops being detected
        if status == "EndOfProcess":
            print("Recording is stopping...")
            recorder.stopMicrophonesRecording()
            speech_recogniser.unsubscribe("speech_recognition")
            eye_color(status)
            break

        # Stop recording when timer is reached
        if timer_cb.is_set():
            recorder.stopMicrophonesRecording()
            speech_recogniser.unsubscribe("speech_recognition")
            leds = ALProxy("ALLeds", ip, port)
            leds.fadeRGB("AllLeds", "White", 1)
            break


def record_audio_sd(timer=None, path_name="/home/nao/microphones/recording.wav", debug=False):
    path_name = os.path.join(path_name)
    # Create Connection Proxies
    recorder = ALProxy("ALAudioRecorder", ip, port)
    sound_detector = ALProxy("ALSoundDetection", ip, port)
    memory = ALProxy("ALMemory", ip, port)
    
    timer_cb = threading.Event()

    sound_detector.subscribe("sound_detector")
    sound_detector.setParameter("Sensitivity", 0.9)

    # Record audio when sound is detected
    if timer is not None:
        threading.Timer(timer, timer_callback, [timer_cb]).start()

    # Start recording
    while True:
        time.sleep(0.5)
        status = memory.getData("SoundDetected")

        # Print status
        if debug:
            print(status)

        # Start recording when sound is detected
        if status is not None:
            print("Recording is starting...")
            recorder.startMicrophonesRecording(path_name, "wav", 16000, (0,0,1,0))
            eye_color("SpeechDetected")
            break

        # Stop recording when timer is reached
        if timer_cb.is_set():
            break

    if timer_cb.is_set():
        return
    
    sound_detector.setParameter("Sensitivity", 0.85)
    while True:
        time.sleep(3)
        status = memory.getData("SoundDetected")

        if debug:
            print("Status: ", status)

        # Stop recording when sound stops being detected
        if status is None:
            print("Recording is stopping...")
            recorder.stopMicrophonesRecording()
            sound_detector.unsubscribe("sound_detector")
            leds = ALProxy("ALLeds", ip, port)
            leds.fadeRGB("AllLeds", "white", 1)
            break

        # Stop recording when timer is reached
        if timer_cb:
            print("Recording is stopping...")
            recorder.stopMicrophonesRecording()
            sound_detector.unsubscribe("sound_detector")
            leds = ALProxy("ALLeds", ip, port)
            leds.fadeRGB("AllLeds", "white", 1)
            break

def reduce_noise(server, amount = 0.5):
    requests.post(server, json={'prop_decrease': amount})

## STT

In [4]:
def convert_wav_to_text(audio_path):
    r = sr.Recognizer()
    with sr.AudioFile(audio_path) as source:
        audio_data = r.record(source)
    try:
        text = r.recognize_google(audio_data)
        return text
    except sr.UnknownValueError:
        return "Speech recognition could not understand audio"
    except sr.RequestError as e:
        return "Could not request results from Google Speech Recognition service: " + str(e)
    except Exception as e:
        return "An error occurred during speech recognition: " + str(e)

## Transfer File

In [10]:
def transfer_file(remote_path="/home/nao/microphones/recording.wav", local_path="recordings/recording.wav"):
    ssh = paramiko.SSHClient()
    ssh.set_missing_host_key_policy(paramiko.AutoAddPolicy())
    ssh.connect(ip, username='nao', password='nao')

    sftp = ssh.open_sftp()
    sftp.get(remote_path, local_path)

    sftp.close()
    ssh.close()
    print("File transfered")

def receive_file(local_path="recordings/recording.wav", remote_path="/home/nao/microphones/recording.wav"):
    ssh = paramiko.SSHClient()
    ssh.set_missing_host_key_policy(paramiko.AutoAddPolicy())
    ssh.connect(ip, username='nao', password='nao')

    sftp = ssh.open_sftp()
    sftp.put(local_path, remote_path)

    sftp.close()
    ssh.close()

## Say Things

In [6]:
import threading
import time

def say(_data):
    tts = ALProxy("ALTextToSpeech", ip, port)
    tts.say(_data)
    return True

def say_sentences(sentences):

    if len(sentences) != 0:
        say(sentences[0])
        # Remove the first sentence
        sentences.pop(0)

    elif len(sentences) != 0 and sentences[0] == "":
        # Remove the first sentence
        sentences.pop(0)

def append_sentences(sentences, sentence):
    sentences.append(sentence)

sentences = ["Hello"]

In [7]:
def say_sentences_thread(sentences):
    if sentences[0] == "quit":
        return

    for i in range(100):
        say_sentences(sentences)
        time.sleep(2)
    print('Stopped')

def append_sentences_thread(sentences):
    while True:
        new_sentence = raw_input("Enter a sentence: ")
        if new_sentence == "quit":
            break
        else:
            sentences.append(new_sentence)

    return sentences

say_thread = threading.Thread(target=say_sentences_thread, args=(sentences,))
append_thread = threading.Thread(target=append_sentences_thread, args=(sentences,))

say_thread.start()
append_thread.start()

say_thread.join()
append_thread.join()

print('Done')

## Think Things

In [7]:
def think(_question):
    url = "http://{}:{}/courseInfo".format(GPT_HOST, GPT_PORT)
    data = {"question": _question}
    r = requests.post(url, json=data)
    return r.json()["answer"]

## MAIN

In [29]:
try:
    while True:
        # translator object initialized during function declaration
        # trigger smart listen
        
        ## if speech detected record up to 15 seconds or less if speech stops
        record_audio_sd(timer=15, debug=False)
        
        # transfer file from pepper to this machine
        transfer_file()

        # pass audio to speech to text and get text back
        reduce_noise()
        text = convert_wav_to_text("recordings/rn_recording.wav")
        print(text)

        # get gpt response
        leds = ALProxy("ALLeds", ip, port)
        leds.fadeRGB("AllLeds", "white", 1)
        # say('Thinking...')
        response = think(text).encode('utf-8')
        
        # pass response to text to speech
        say(response)
        # say('One flow completed.')
        
    
except KeyboardInterrupt:
    recorder = ALProxy("ALAudioRecorder", ip, port)
    recorder.stopMicrophonesRecording()
    leds = ALProxy("ALLeds", ip, port)
    leds.fadeRGB("AllLeds", "white", 1)

Recording is starting...
Recording is stopping...
File transfered
Noise reduce at 10%
Timer reached
hello Peppa tell me about any courses which you have on AI cos I'm really interested in the eyes
Noise reduce at 15%
tell me about any courses that you have on why cos I'm really interested in the eyes
Noise reduce at 20%
tell me about any courses that you have on why cos I'm really interested in the eyes
Noise reduce at 25%
hello Peppa tell me about any courses which you have on AI cos I'm really interested in the eyes
Noise reduce at 30%
hello Peppa tell me about any courses which you have on a i cos I'm really interested in the eyes
Noise reduce at 35%
hello Peppa tell me about any courses which you have on a i cos I'm really interested in
Noise reduce at 40%
hello Peppa tell me about any courses which you have on a i cos I'm really interested in
Noise reduce at 45%
hello Peppa tell me about any courses which you have on AI cos I'm really interested in
Noise reduce at 50%
hello Peppa 

In [15]:
recorder = ALProxy("ALAudioRecorder", ip, port)
recorder.stopMicrophonesRecording()
leds = ALProxy("ALLeds", ip, port)
leds.fadeRGB("AllLeds", "white", 1)

In [30]:
import requests
from json import dumps

i = 0.1
while i < 1:
    print 'Noise reduce at ' + str(int(round(i*100))) + '%'
    requests.post("http://10.109.16.45:8891/noise", json={"prop_decrease": i})
    text = convert_wav_to_text("recordings/rn_recording.wav")
    print(text)
    i += 0.1

Noise reduce at 10%
hello Peppa can you give me some courses on business about exit
Noise reduce at 20%
hello Peppa can you give me some courses on business about exit
Noise reduce at 30%
hello Peppa can you give me some courses on business about exit
Noise reduce at 40%
hello Peppa can you give me some courses on business about exit
Noise reduce at 50%
hello Peppa can you give me some courses on business about exit
Noise reduce at 60%
hello Peppa can you give me some courses on business about exit
Noise reduce at 70%
hello Peppa can you give me some courses on Facebook about exit
Noise reduce at 80%
hello Peppa can you give me some courses on Facebook about exit
Noise reduce at 90%
hello Peppa can you give me some courses on business about exit
Noise reduce at 100%
hello can you


In [33]:
memory = ALProxy('ALMemory', ip, port)
memory.getDataListName()

['Dialog/UnloadTopic',
 'ALSignsAndFeedback/Speaking',
 'BootConfig/HasStartedAtBoot',
 'AutonomousLife/Asleep',
 'AnimatedRecharge/WarningMode',
 'ALSignsAndFeedback/Switched',
 'AppLauncher/CustomLauncher',
 'ALRALManagerModule/onTouchDown',
 'ALRecharge/LookFalsePositive',
 'ALRecharge/LookPreviousHypothesis',
 'ALRecharge/SearchStopped',
 'ALRecharge/LeaveFailed',
 'ALRecharge/DockingSuccess',
 'ALRecharge/StationNotFound',
 'ALRecharge/StationDetected',
 'Dialog/MyRobotName',
 'Dialog/StartApp',
 'Dialog/Second',
 'Dialog/Minute',
 'Dialog/Hour',
 'Dialog/Month',
 'ALRecharge/LookForStationSuccess',
 'Dialog/Year',
 'Dialog/Threshold/SLM',
 'Dialog/NothingToSay',
 'Dialog/Default',
 'Dialog/RobotName',
 'Dialog/SameRule',
 'Dialog/TalkTime',
 'Dialog/SaidMisunderstood',
 'Dialog/NotUnderstoodEvent',
 'Dialog/NoOneSpeak5',
 'Dialog/Fallback',
 'Dialog/NotSpeaking20',
 'Dialog/NotSpeaking15',
 'Dialog/NotSpeaking10',
 'Dialog/NotUnderstood',
 'Dialog/RobotIP',
 'Dialog/LastInput',
 

In [40]:
tracker = ALProxy("ALTracker", ip, port)
tracker.registerTarget("Face", 0.1)
tracker.setMode("Head")
tracker.track("Face")

for i in range(10):
    time.sleep(3)
    print(tracker.getTargetPosition(0))

None
[0.40254080295562744, -0.21153926849365234, 0.5685097575187683]
None
[0.9793750643730164, -0.23001500964164734, 0.831806480884552]
None
[0.9793752431869507, -0.2300148606300354, 0.831806480884552]
None
[1.234731674194336, -0.20777323842048645, 0.7367026805877686]
None
[0.6246371269226074, -0.08070826530456543, 0.7516487240791321]
None
[0.7460576295852661, -0.07584789395332336, 0.6862739324569702]
None
[0.7367236018180847, -0.07449603080749512, 0.694361686706543]
None
[0.2506811022758484, -0.017724335193634033, 0.6908724308013916]
None
[0.4728855490684509, -0.09135261178016663, 0.8757207989692688]
None
[0.4708067774772644, -0.0913446843624115, 0.8763599991798401]


In [54]:
memory = ALProxy("ALMemory", ip, port)
gaze = ALProxy('ALGazeAnalysis', ip, port)
gaze.setTolerance(0.1)

for i in range(10):
    time.sleep(3)
    person_id = memory.getData('PeoplePerception/PeopleList')
    if len(person_id) != 0:
        print(memory.getData('GazeAnalysis/PeopleLookingAtRobot'))

None
None
None
None
None
None
None
None
